## 03- Entrenamiento del Modelo

En este notebook realizaremos los procedimientos necesarios para entrenar el modelo que permita establecer la probabilidad de que un cliente no renueve su producto con la compañía, basado en las diferentes dimensiones de información que se tengan del mismo al momento de emitir la póliza y que hemos explorado en los notebooks anteriores. Para ello usaremos las ventajas de la librería `scikit-learn`.

In [1]:
from model_parameters import *
import pandas as pd
import numpy as np

data=pd.read_csv("data_modelacion.csv",sep=";",dtype={'Asegurado__c':str,'NumeroPoliza__c':str,'CodigoTipoAsegurado__c':str,'churn':int,'ClaseVehiculo__c':str,'TipoVehiculo__c':str,'PuntoVenta__c':str,'MarcaVehiculo__c':str})

## Generar Particiones de Datos

Se debe generar una partición de la data para test(10%), el restante se usará para entrenamiento y validación.

In [2]:
#Definir esquema para valores faltantes
num_cols=[i for i in data.columns.values.tolist() if data[i].dtype!="O" and i not in ['churn','Profesion__pc','Asegurado__c','CodigoTipoAsegurado__c','PuntoVenta__c','RamoTecnico__c','Tipo_poliza_c']]
str_cols=[i for i in data.columns.values.tolist() if data[i].dtype not in ["int32","float","int64"] and i not in ['churn','NumeroPoliza__c','Producto__c','Asegurado__c','CodigoTipoAsegurado__c','PuntoVenta__c','RamoTecnico__c','Tipo_poliza_c']]
train,test=train_test_split(data, test_size=0.10,random_state=123)

## Modelos a Evaluar

Se procede a evaluar dos tipos de modelos de tipo boosting (XGboost,LightGBM), los cuales son ampliamente usados en el modelamiento predictivo. A su vez, se prueba un modelo sencillo como la regresión logística, para comparar el desempeño frente a modelos de mayor complejidad. El objetivo general de esta comparación es establecer qué tipo de modelo puede encontrar los aspectos más relevantes para establecer cuándo un cliente tiene mayor propensión a renovar su producto con la compañía.

In [3]:
from sklearn import set_config
set_config(display='diagram')

In [4]:
X_train,Y_train=train.drop(columns=["churn","Asegurado__c"]),train['churn'].values

In [5]:
## Pasos de imputación y estandarización
num_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="constant",fill_value=0)), ("scaler", StandardScaler())]
)

def get_classifier_model(model_name:str,k_cv:int=5,scoring="roc_auc") -> GridSearchCV:
    """retorna un objeto GridSearchCV para entrenar con @param model_name y k_cv-fold Cross Validation
    para clasificación binaria, la métrica de scoring de los modelos está dada por @param scoring
    """
    cat_transformer = OneHotEncoder(handle_unknown="error")
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", num_transformer, num_cols),
            ("cat", cat_transformer, str_cols),
        ],sparse_threshold=0
    )

    model=MODEL_LIST[model_name]
    pipe=Pipeline([
        ('cleaner',preprocessor),
        ('pca',PCA()),
        ('model',model)

    ], verbose=True)

    param_grid=MODEL_PARAMS[model_name]['param_grid']

    grid=GridSearchCV(pipe,param_grid,cv=k_cv,refit=True,scoring=scoring,verbose=True)
    return grid

xgb=get_classifier_model("xgboost")
logistic=get_classifier_model("logistic_regression")
lightgbm=get_classifier_model("lightgbm")

In [6]:
xgb.fit(X_train,Y_train.ravel())

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   0.1s
[Pipeline] ............. (step 3 of 3) Processing model, total=   0.6s
[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   0.1s
[Pipeline] ............. (step 3 of 3) Processing model, total=   0.7s
[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   0.1s
[Pipeline] ............. (step 3 of 3) Processing model, total=   0.7s
[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   0.1s
[Pipeline] ............. (step 3 of 3) Processing model, total=   0.7s
[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ..

KeyboardInterrupt: 

In [7]:
logistic.fit(X_train,Y_train.ravel())

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   0.1s
[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   0.1s
[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   0.1s
[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   0.1s
[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   0.1s
[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   0.1s
[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ......

C:\Users\jhonp\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\jhonp\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\jhonp\miniconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\jhonp\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1471, in fit
    raise ValueError(
ValueError: l1_ratio must be

[Pipeline] ........... (step 1 of 3) Processing cleaner, total=   0.1s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   0.1s


ValueError: l1_ratio must be between 0 and 1; got (l1_ratio=None)

In [15]:
lightgbm.fit(X_train,Y_train.ravel())


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[Pipeline] ........... (step 1 of 2) Processing cleaner, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s
[Pipeline] ........... (step 1 of 2) Processing cleaner, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s
[Pipeline] ........... (step 1 of 2) Processing cleaner, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s
[Pipeline] ........... (step 1 of 2) Processing cleaner, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s
[Pipeline] ........... (step 1 of 2) Processing cleaner, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s
[Pipeline] ........... (step 1 of 2) Processing cleaner, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.3s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('cleaner',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value=0,
                                                                                                        strategy='constant')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['n_prod_prev',
                                                                          'total_siniestros',
                                                                          'total_pagado_smmlv',
                                                                          'anios_ultimo_siniestro',
                                                                          'Activos__c',
                                                                          'AnnualRevenue',
                                                                          'MontoAnual__c',
                                                                          'OtrosIngresos__c',
                                                                          'Egres...
                                                                          'EstadoCivil__pc',
                                                                          'Genero__pc'])])),
                                       ('model', LGBMClassifier())],
                                verbose=True),
             param_grid={'model__colsample_bytree': [0.8],
                         'model__max_depth': [15],
                         'model__min_split_gain': [0.3],
                         'model__n_estimators': [100],
                         'model__num_leaves': [50], 'model__reg_alpha': [1.1],
                         'model__reg_lambda': [1.1], 'model__subsample': [0.9],
                         'model__subsample_freq': [20]},
             scoring='roc_auc', verbose=True)

In [74]:
display(pipe)

Pipeline(steps=[('cleaner',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['n_prod_prev',
                                                   'total_siniestros',
                                                   'total_pagado_smmlv',
                                                   'anios_ultimo_siniestro',
                                                   'Activos__c',
                                                   'AnnualRevenue',
                                                   'MontoAnual__c',
                                                   'OtrosIngresos__c',
                                                   'EgresosAnuales__c',
                                                   'edad']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['tipo_poliza_name',
                                                   'tipo_prod_desc',
                                                   'ClaseVehiculo__c',
                                                   'TipoVehiculo__c',
                                                   'EstadoCivil__pc',
                                                   'Genero__pc'])])),
                ('model', GradientBoostingClassifier())],
         verbose=True)

In [19]:
test['churn'].values.ravel()

array([1, 1, 0, ..., 1, 0, 0])

In [18]:
metrics(test['churn'].values.ravel(),xgb.predict(test.drop(columns="churn")))

 Accuracy:0.8404 
 Precision: 0.0000 
 Recall: 0.0000 
 Kappa: 0.0000 
 F1-Score: 0.0000 


C:\Users\jhonp\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'kappa': 0.0,
 'accuracy': 0.84039745381152,
 'f1': 0.0,
 'prec': 0.0,
 'recall': 0.0}

In [29]:
from utils import metrics

print("===Métricas para XGboost===\n")
print("\n\t Training\n")
_=metrics(Y_train.ravel(),xgb.predict(X_train))
print("\n\t Test \n")
_=metrics(test['churn'].values.ravel(),xgb.predict(test.drop(columns="churn")))



===Métricas para XGboost===


	 Training

 Accuracy:0.8392 
 Precision: 0.0000 
 Recall: 0.0000 
 Kappa: 0.0000 
 F1-Score: 0.0000 

	 Test 

 Accuracy:0.8404 
 Precision: 0.0000 
 Recall: 0.0000 
 Kappa: 0.0000 
 F1-Score: 0.0000 


C:\Users\jhonp\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jhonp\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
